# Fault Injection on Quantized Neural Networks

This notebook tests fault injection on quantized neural networks modified to mitigate the impact of faults.

The different precision neural networks inspired at VGG-16 feature 6 convolutional layers, 3 max pool layers and 3 fully connected layers. The networks classify the CIFAR-10 testset. The 2 different precision networks tested here are:

- CNVW1A1 using 1 bit weights and 1 bit activation,
- CNVW1A2 using 1 bit weights and 2 bit activation,
- CNVW2A2 using 2 bit weights and 2 bit activation

## 1. Import the packages

In [ ]:
import bnn
import os
import csv
from copy import deepcopy
from json import dumps
from PIL import Image
from yapf.yapflib.yapf_api import FormatCode
import scipy.io as sio
import matplotlib.pyplot as plt
from pynq import Xlnk

## 2. Load the required datasets

This notebook requires the CIFAR-10, GTSRB, and SVHN datasets. You can download them from each given url via wget and unzip it to a folder on Pynq as shown below.

In [ ]:
def load_cifar10_testset(path):
    if not os.path.exists(path + "/test_batch_1000.bin"):
        with open(path + "/test_batch.bin", "rb") as infile:
            with open(path + "/test_batch_1000.bin", "wb+") as outfile:
                for i in range(1000):
                    outfile.write(infile.read(3073))

    labels = []
    input_file = path + "test_batch_1000.bin"
    with open(input_file, "rb") as file:
        #for 10000 pictures
        for i in range(1000):
            #read first byte -> label
            labels.append(int.from_bytes(file.read(1), byteorder="big"))
            #read image (3072 bytes) and do nothing with it
            file.read(3072)
        file.close()

    print("Read", len(labels), "labels")
    return (input_file, labels)


if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/cifar-10-binary.tar.gz"):
    #get
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/cifar-10-batches-bin/"):
    #unzip
    !tar -xf cifar-10-binary.tar.gz

cifar_files, cifar_labels = load_cifar10_testset("/home/xilinx/jupyter_notebooks/bnn/cifar-10-batches-bin/")

In [ ]:
def load_gtsrb_testset(path):
    image_files = []
    images = []
    labels = []

    with open(path + "/GT-final_test.csv") as gtfile:
        gtreader = csv.reader(gtfile, delimiter=';')
        next(gtreader)
        for row in gtreader:
            image_files.append(path + '/' + row[0])
            labels.append(int(row[7]))

    labels = labels[0:1000]
    for i in range(1000):
        images.append(Image.open(image_files[i]))
        images[i].load()

    return (images, labels)


if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB_Final_Test_Images.zip"):
    !wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB_Final_Test_GT.zip"):
    !wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB"):
    !unzip -q -o GTSRB_Final_Test_Images.zip
    !unzip -q -o GTSRB_Final_Test_GT.zip
    !mv GT-final_test.csv GTSRB/Final_Test/Images

gtsrb_files, gtsrb_labels = load_gtsrb_testset("/home/xilinx/jupyter_notebooks/bnn/GTSRB/Final_Test/Images")

In [ ]:
def load_svhn_testset(path):
    images = []
    labels = []

    data = sio.loadmat(path)

    # Subtact 1 to match classifier output. E.g. The classifier will return class 1 for an image of the number 2.
    label_mat = data['y'] - 1
    image_mat = data['X']

    labels = label_mat.transpose().tolist()[0]
    labels = labels[0:1000]

    #for i in range(image_mat.shape[3]):
    for i in range(1000):
        images.append(Image.fromarray(image_mat[:,:,:,i]))
        images[i].load()

    return (images, labels)


if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/test_32x32.mat"):
    !wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

svhn_files, svhn_labels = load_svhn_testset("/home/xilinx/jupyter_notebooks/bnn/test_32x32.mat")

&nbsp;
## 3. Start Xlnk Interface

In [ ]:
xlnk = Xlnk()

&nbsp;
## 4. A Simple Fault Test Example
This example demonstrates the most simple method of running a fault test, and demonstrates the use of all of the classification function arguments. The classes beyond this section expand on this concept and allow the user to more easily set up customizeable and comprehensive tests.

Non-CIFAR10 datasets (e.g. GTSRB and SVHN) will need to be converted to CIFAR-10 format before being classified. This will be automatically handled by using the `CnvClassifier.classify_images()` function instead of `CnvClassifier.classify_cifars()`. This function takes a list of Pillow Images and converts them to the required format before classifying. For fault injection, use `CnvClassifier.classify_images_with_faults()`.

The inference can be performed with different precision for weights and activation. Creating a specific Classifier will automatically download the correct bitstream onto PL and load the weights and thresholds trained on the specific dataset.

In [ ]:
print("Availabe params:", bnn.available_params(bnn.NETWORK_CNVW1A1))

# Instantiate the cnvW1A1 hardware classifier with the CIFAR-10 dataset
classifier = bnn.CnvClassifier(bnn.NETWORK_CNVW1A1, 'cifar10', bnn.RUNTIME_HW)

# Classify the dataset, and inject 10000 MBUs, targeting both weights and thresholds in the first 3 layers.
#
# The default value of target_layers is an empty list, which will target all layers.
#
# target_type determines if weights or thresholds will be targeted. A value of -1 causes both to be targeted, a value
# of 0 causes weights to be targeted, and a value of 1 will target thresholds.
print("Classifying", len(cifar_labels), "images with", classifier.net + "-" + classifier.params)
results = classifier.classify_cifars_with_faults(cifar_files, num_faults=10000, flip_word=True, target_type=-1, target_layers=[0, 1, 2])

# Reset the device
xlnk.xlnk_reset()

# Calculate the accuracy of the network
countRight = 0
for i in range(len(cifar_labels)):
    if labels[i] == results[i]:
        countRight += 1
accuracy = (countRight * 100) / len(labels)

print("Accuracy:", accuracy)

&nbsp;
## 5. Define Utility Functions

Various functions for handling dictionaries and calculating statistics

In [ ]:
def calculate_accuracy(results, labels):
    countRight = 0
    for idx in range(len(labels)):
        if labels[idx] == results[idx]:
            countRight += 1
    return countRight*100/len(labels)


#Merge dictionaries that contain sub-dictionaries
def dict_of_dicts_merge(*dicts):
    out = {}
    for item in dicts:
        overlapping_keys = out.keys() & item.keys()
        for key in overlapping_keys:
            if (isinstance(out[key], dict) and isinstance(item[key], dict)):
                out[key] = dict_of_dicts_merge(out[key], item[key])
        for key in item.keys() - overlapping_keys:
            out[key] = deepcopy(item[key])
    return out


#Output dict contains the combined raw results from a test. Used as input to calculate_stats
def make_output_dict(network_name, dataset, num_runs, num_flips, accuracy_control, run_type, accuracies):
    out = {}
    out["network"] = network_name
    out["dataset"] = dataset
    out["run count"] = num_runs
    out["flips"] = num_flips
    out["control"] = accuracy_control
    out["results"] = {}
    out["results"][run_type] = accuracies
    return out


#Calculates various stats given the results of a test (formatted with make_output_dict)
def calculate_stats(output_dict):
    out = output_dict.copy()
    for key, value in output_dict["results"].items():
        out["results"][key] = {}
        out["results"][key]["runs"] = {}
        out["results"][key]["runs"]["all"] = value
        out["results"][key]["runs"]["effective"] = list(filter(lambda x: x != out["control"], value))
        out["results"][key]["effective count"] = len(out["results"][key]["runs"]["effective"])
        out["results"][key]["min accuracy"] = min(value)
        out["results"][key]["max accuracy"] = max(value)
        if out["results"][key]["effective count"] != 0:
            out["results"][key]["avg accuracy"] = sum(value) / len(value)
            out["results"][key]["avg effective accuracy"] = sum(out["results"][key]["runs"]["effective"]) / out["results"][key]["effective count"]
            out["results"][key]["accuracy delta"] = out["control"] - out["results"][key]["avg accuracy"]
            out["results"][key]["effective accuracy delta"] = out["control"] - out["results"][key]["avg effective accuracy"]
        else:
            out["results"][key]["avg accuracy"] = out["control"]
    return out


def dict_to_str(dict):
    dict_string = dumps(dict)
    formatted_code, _ = FormatCode(dict_string)
    return formatted_code


def write_stats_file(file_name, stats_dict):
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    f = open(file_name, "w+")
    f.write(dict_to_str(stats_dict))
    f.close()

&nbsp;

## 6. Define Fault Testing Classes

### Fault Test Class
Uses the given network type and image set to run a series of fault tests on the network. The network is reset between runs.

**Constructor Arguments:**
- **network:** The network to test. e.g. bnn.NETWORK_CNVW1A1
- **dataset:** The dataset that's being tested. (`'cifar10'`, `'streetview'`, or `'road-signs'`)
- **input_file:** The file, or list of Images for non-cifar tests, containing the images to be tested.
- **labels:** The list of ground truth labels for calculating accuracy.
- **control_accuracy:** Optional parameter specifying the control accuracy of the network. Will be calculated if not provided.

**FaultTest.run_test() Arguments:**
- **num_runs:** The number of tests to perform
- **num_flips:** The number of faults to inject per run
- **flip_word:** A boolean indicating if a bit or word should be flipped
- **weight_or_threshold:** An integer specifying if weights, thresholds, or both should be targeted.

        -1 = target weights and activations
        0  = target weights only
        1  = target activations only
- **target_layers:** An array of integers specifying which layers to target. Leave empty to target all layers.

In [ ]:
class FaultTest:
    def __init__(self, network, dataset, input_file, labels):
        self.network = network
        self.dataset = dataset
        self.input_file = input_file
        self.labels = labels

    @classmethod
    def CIFARTest(cls, network, input_file, labels):
        return cls(network, 'cifar10', input_file, labels)

    @classmethod
    def SVHNTest(cls, network, input_file, labels):
        return cls(network, 'streetview', input_file, labels)

    @classmethod
    def GTSRBTest(cls, network, input_file, labels):
        return cls(network, 'road-signs', input_file, labels)
    
    def run_test(self, num_runs, num_flips, flip_word=False, weight_or_threshold=-1, target_layers = []):
        results    = [ None for i in range(num_runs) ]
        times      = [ None for i in range(num_runs) ]
        accuracies = [ None for i in range(num_runs) ]

        for i in range(num_runs):
            hw_classifier = bnn.CnvClassifier(self.network, self.dataset, bnn.RUNTIME_HW)

            message = "{}-{} run {} of {}".format(self.network, self.dataset, i+1, num_runs)
            if len(target_layers) == 0:
                message += " (flipping {} {}(s) in any layer)".format(num_flips, "word" if flip_word else "bit")
            else:
                message += " (flipping {} {}(s) in layer(s) {})".format(num_flips, "word" if flip_word else "bit", target_layers)
            print(message)

            if self.dataset == 'cifar10':
                results[i] = hw_classifier.classify_cifars_with_faults(self.input_file, num_flips, flip_word, weight_or_threshold, target_layers)
            else:
                results[i] = hw_classifier.classify_images_with_faults(self.input_file, num_flips, flip_word, weight_or_threshold, target_layers)

            times[i]      = hw_classifier.usecPerImage
            accuracies[i] = calculate_accuracy(results[i], self.labels)

            print("Accuracy:", accuracies[i])

            xlnk.xlnk_reset()
            print()

        return (results, times, accuracies)

&nbsp;

### Network Test Class
Takes a FaultTest object and use it to run a comprehensive series of tests on different combinations of targets and SEU/MBU

**Constructor Arguments:**
- **fault_test:** A FaultTest object
- **output_folder:** The folder to store the test results in

**NetworkTest.test_network() Arguments:**
- **num_runs:** The number of times to run each test
- **num_flips:** The number of faults to inject each run
- **bit_flips:** Whether or not to run bit flip tests
- **word_flips:** Whether or not to run word flip tests
- **weights:** Whether or not to run tests targeting weights
- **thresholds:** Whether or not to run tests targeting thresholds
- **target_layers:** An array of integers specifying which layers to target. Leave empty to target all layers.

In [ ]:
class NetworkTest:
    def __init__(self, fault_test, output_folder):
        self.fault_test = fault_test
        self.output_folder = output_folder + '/' + self.fault_test.network + '/' + self.fault_test.dataset + '/'
        self.control = -1


    def __run_control(self):
        _, _, accuracy = self.fault_test.run_test(num_runs=1, num_flips=0);
        self.control = accuracy[0]


    def __build_output_dict(self, name, num_runs, num_flips, accuracies):
        return make_output_dict(
            self.fault_test.network,
            self.fault_test.dataset,
            num_runs,
            num_flips,
            self.control,
            name,
            accuracies
        )


    def __run_tests(self, folder, num_runs, num_flips, bit_flips, word_flips, weights, thresholds, target_layers):
        #Separate output dicts for each run. They will be combined later for the
        #final results, and the redundant data will be discarded.
        weight_bit_output = {}
        threshold_bit_output = {}
        weight_word_output = {}
        threshold_word_output = {}
        stats = {}

        if bit_flips:
            if weights: #Test bit flips (weights)
                _, _, accuracies = self.fault_test.run_test(num_runs, num_flips, False, 0, target_layers)
                
                weight_bit_output = self.__build_output_dict("weight bit", num_runs, num_flips, accuracies)

                write_stats_file(folder + "/temp/" + self.fault_test.network + "_results_bit_weight.json", weight_bit_output)

            if thresholds: #Test bit flips (thresholds)
                _, _, accuracies = self.fault_test.run_test(num_runs, num_flips, False, 1, target_layers)
                
                threshold_bit_output = self.__build_output_dict("threshold bit", num_runs, num_flips, accuracies)

                write_stats_file(folder + "/temp/" + self.fault_test.network + "_results_bit_threshold.json", threshold_bit_output)

        if word_flips:
            if weights: #Test word flips (weights)
                _, _, accuracies = self.fault_test.run_test(num_runs, num_flips, True, 0, target_layers)
                
                weight_word_output = self.__build_output_dict("weight word", num_runs, num_flips, accuracies)

                write_stats_file(folder + "/temp/" + self.fault_test.network + "_results_word_weight.json", weight_word_output)
            
            if thresholds: #Test word flips (thresholds)
                _, _, accuracies = self.fault_test.run_test(num_runs, num_flips, True, 1, target_layers)
                
                threshold_word_output = self.__build_output_dict("threshold word", num_runs, num_flips, accuracies)

                write_stats_file(folder + "/temp/" + self.fault_test.network + "_results_word_threshold.json", threshold_word_output)

        # Calculate stats
        stats = dict_of_dicts_merge(weight_bit_output, threshold_bit_output, weight_word_output, threshold_word_output)
        stats = calculate_stats(stats)
        return stats


    def test_network(self, num_runs, num_flips, bit_flips=True, word_flips=True, weights=True, thresholds=True, target_layers = []):
        folder = self.output_folder + '/' + str(num_flips) + 'flips/'

        # Calculate the control accuracy
        if self.control < 0:
            self.__run_control()

        # Run all tests
        stats = self.__run_tests(folder, num_runs, num_flips, bit_flips, word_flips, weights, thresholds, target_layers)

        # Write all stats to the output file
        filename = folder + '/' + self.fault_test.network + '_' + self.fault_test.dataset
        if len(target_layers) > 0:
            filename = filename + "_stats_layer{}.json".format(target_layers)
        else:
            filename = filename + "_stats.json"

        write_stats_file(filename, stats)

&nbsp;

## 7. Run the tests
The dictionary below instantiates tests for all of the available CNV networks. These tests are then conducted for each individual layer multiple times, with an increasing number of faults each time.

In [ ]:
output_folder = "/home/xilinx/jupyter_notebooks/bnn/faults/"

num_runs = 40
flip_counts = [5, 10, 50, 100]

tests = {
    # cnvW1A1 Networks
    'cnvW1A1': [
        NetworkTest(FaultTest.CIFARTest(bnn.NETWORK_CNVW1A1, cifar_files, cifar_labels), output_folder),
        NetworkTest(FaultTest.SVHNTest(bnn.NETWORK_CNVW1A1, svhn_files, svhn_labels), output_folder),
        NetworkTest(FaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1, gtsrb_files, gtsrb_labels), output_folder),
    ],
    'cnvW1A1-TMR': [
        NetworkTest(FaultTest.CIFARTest(bnn.NETWORK_CNVW1A1_TMR, cifar_files, cifar_labels), output_folder),
        NetworkTest(FaultTest.SVHNTest(bnn.NETWORK_CNVW1A1_TMR, svhn_files, svhn_labels), output_folder),
        NetworkTest(FaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1_TMR, gtsrb_files, gtsrb_labels), output_folder),
    ],
    'cnvW1A1-interleaved': [
        NetworkTest(FaultTest.CIFARTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, cifar_files, cifar_labels), output_folder),
        NetworkTest(FaultTest.SVHNTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, svhn_files, svhn_labels), output_folder),
        NetworkTest(FaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, gtsrb_files, gtsrb_labels), output_folder),
    ],

    # cnvW1A2 Networks
    'cnvW1A2': [
        NetworkTest(FaultTest.CIFARTest(bnn.NETWORK_CNVW1A2, cifar_files, cifar_labels), output_folder)
    ],
#    'cnvW1A2-TMR': [
#        NetworkTest(FaultTest.CIFARTest(bnn.NETWORK_CNVW1A2_TMR, cifar_files, cifar_labels), output_folder)
#    ],
    'cnvW1A2-interleaved': [
        NetworkTest(FaultTest.CIFARTest(bnn.NETWORK_CNVW1A2_INTERLEAVED, cifar_files, cifar_labels), output_folder)
    ],

    # cnvW2A2 Networks
    'cnvW2A2': [
        NetworkTest(FaultTest.CIFARTest(bnn.NETWORK_CNVW2A2, cifar_files, cifar_labels), output_folder)
    ],
    'cnvW2A2-TMR': [
        NetworkTest(FaultTest.CIFARTest(bnn.NETWORK_CNVW2A2_TMR, cifar_files, cifar_labels), output_folder)
    ],
    'cnvW2A2-interleaved': [
        NetworkTest(FaultTest.CIFARTest(bnn.NETWORK_CNVW2A2_INTERLEAVED, cifar_files, cifar_labels), output_folder)
    ],
}

In [ ]:
# Run all tests
for layer in range(9):
    for flips in flip_counts:
        for key, test_array in tests.items():
            for test in test_array:
                test.test_network(num_runs, flips, bit_flips=True, word_flips=True, weights=True, thresholds=True, target_layers=[layer])